In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import requests

In [9]:
# access trading currency pairs via MEXC api

curl = f"https://contract.mexc.com/api/v1/contract/detail/"
response = requests.get(curl)
data = pd.DataFrame(response.json()['data'])
result = pd.DataFrame()
result['currency pair'] = data['symbol'].astype(str)
result[['position_limit_for_min_leverage', 'max_leverage', 'position_limit_for_max_leverage']] = pd.DataFrame([[None, None, None]], index=result.index)

In [10]:
# reject cookies

def reject_cookies(driver, reject_xpath):
    try:
        reject_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, reject_xpath))
        )
        reject_button.click()
    except Exception as e:
        print(f"An error occurred while accepting cookies: {e}")

reject_xpath = "/html/body/div[2]/div[2]/div/div[1]/div/div[2]/div/button[2]"

In [11]:
# search currency pairs and scrape infomation

def click_input(driver, click_xpath, input_xpath, test_xpath, target_xpath ,result):
    # click
    click_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, click_xpath))
        )
    click_button.click()
    # input
    input_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, input_xpath))
    )
    for currency in result["currency pair"]:
        currency_binance = currency.replace("_","")
        input_box.clear()
        input_box.send_keys(currency_binance)
        input_box.send_keys(Keys.RETURN)
        # check whether the result matches the input
        test = driver.find_element(By.XPATH,test_xpath)
        value = test.get_attribute("value")
        value = str(value)
        if value == currency_binance:
            # extract
            position_limit_for_max_leverage = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, target_xpath["position_max_xpath"]))
            )
            max_leverage = driver.find_element(By.XPATH, target_xpath["max_leverage_xpath"])
            position_limit_for_min_leverage = driver.find_element(By.XPATH, target_xpath["position_min_xpath"])
            # position_limit_for_max_leverage
            position_limit_for_max_leverage = position_limit_for_max_leverage.text
            max_parts = position_limit_for_max_leverage.split('-')
            position_limit_for_max_leverage = max_parts[1].strip()
            # max_leverage
            max_leverage = max_leverage.text
            # position_limit_for_min_leverage
            position_limit_for_min_leverage = position_limit_for_min_leverage.text
            min_parts = position_limit_for_min_leverage.split('-')
            position_limit_for_min_leverage = min_parts[1].strip()
            # load
            result.loc[result["currency pair"]==currency,"position_limit_for_min_leverage"]=position_limit_for_min_leverage
            result.loc[result["currency pair"]==currency,"max_leverage"]=max_leverage
            result.loc[result["currency pair"]==currency,"position_limit_for_max_leverage"]=position_limit_for_max_leverage
        else: 
            result.loc[result["currency pair"]==currency,"position_limit_for_min_leverage"]='NA'
            result.loc[result["currency pair"]==currency,"max_leverage"]='NA'
            result.loc[result["currency pair"]==currency,"position_limit_for_max_leverage"]='NA'
        time.sleep(0.1)
        click_button.click()

click_xpath = "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[1]"
input_xpath = "//html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[2]/div/div/input"
test_xpath = "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/input"
target_xpath = {
    "position_max_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[1]/td[2]",
    "max_leverage_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[1]/td[3]",
    "position_min_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[last()]/td[2]"
}

In [12]:
def main(url):
    driver = webdriver.Chrome()   
    try:
        driver.get(url)
        reject_cookies(driver, reject_xpath)
        click_input(driver, click_xpath, input_xpath, test_xpath, target_xpath, result)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit() 

In [13]:
url = "https://www.binance.com/en/futures/trading-rules/perpetual/leverage-margin"
main(url)

In [ ]:
result.to_csv("Leverage Information.csv")